In [1]:
# Imports and variables
import os

from analysis import compute_regression
from analysis import trim_5_percentiles

import pandas as pd
import numpy as np
import analytics

import logging
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.ERROR)

In [2]:
DUMMY_RUN = False  # Change this to False when the analysis is run for real
ANALYTICS_VIEW_ID = '101677264'
GBQ_PROJECT_ID = '620265099307'
%autosave 0

Autosave disabled


# Engagement outcomes

In [3]:
# Import page views data from csv
df1 = pd.read_csv(os.path.join('..','data','page_views_ccg.csv'),usecols={"Page","Date","Pageviews","Unique Pageviews"} )
dfp = pd.read_csv(os.path.join('..','data','page_views_practice.csv'),usecols={"Page","Date","Pageviews","Unique Pageviews"} ) 
all_stats = pd.concat([df1,dfp])
all_stats.head()

,Page,Date,Pageviews,Unique Pageviews
0,/ccg/02P/measures/?tags=lowpriority,20180918,23,1
1,/ccg/99Q/measures/?tags=lowpriority,20180926,22,4
2,/ccg/99P/measures/?tags=lowpriority,20180926,18,3
3,/ccg/10V/measures/?tags=lowpriority,20181005,16,1
4,/ccg/03R/measures/?tags=lowpriority,20181114,15,8


In [4]:
# extract if ccg/practice code from path
all_stats["org_id"] = np.where(
    all_stats.Page.str.contains("ccg"),
    all_stats.Page.str.replace('/ccg/', '').str[:3],
    all_stats.Page.str.replace('/practice/', '').str[:6])
all_stats["org_type"] = np.where(
    all_stats.Page.str.contains("ccg"),
    "ccg",
    'practice')

#convert dates to date format
all_stats["Date"] = pd.to_datetime(all_stats["Date"], format='%Y%m%d')

all_stats.head(2)

,Page,Date,Pageviews,Unique Pageviews,org_id,org_type
0,/ccg/02P/measures/?tags=lowpriority,2018-09-18,23,1,02P,ccg
1,/ccg/99Q/measures/?tags=lowpriority,2018-09-26,22,4,99Q,ccg


In [5]:
### CCGs that have been allocated to the RCT 
rct_ccgs = pd.read_csv(os.path.join('..','data','randomisation_group.csv'))

# Joint Team information (which CCGs work together in Joint Teams)
team = pd.read_csv(os.path.join('..','data','joint_teams.csv'))

# Map CCGs to Joint Teams
rct_ccgs = rct_ccgs.merge(team, on="joint_team", how="left")

# Fill blank ccg_ids from joint_id column, so even CCGs not in Joint Teams 
# have a value for joint_id
rct_ccgs["pct_id"] = rct_ccgs["ccg_id"].combine_first(rct_ccgs["joint_id"])
rct_ccgs = rct_ccgs[["joint_id", "allocation", "pct_id"]]

# Add numerical intervention field
rct_ccgs['intervention'] = rct_ccgs.allocation.map({'con': 0, 'I': 1})

rct_ccgs.head(2)

,joint_id,allocation,pct_id,intervention
0,01X,con,01X,0
1,99K,con,99K,0


In [6]:
## Map practices to joint teams, for practice-level analysis

# Get current mapping data from bigquery
practice_to_ccg = '''select distinct ccg_id, code
from `ebmdatalab.research.practices_2019_09`
where setting = 4 and status_code != 'C'
'''

practice_to_ccg = pd.read_gbq(practice_to_ccg, GBQ_PROJECT_ID, dialect='standard')
practice_to_ccg.to_csv(os.path.join('..','data','practice_to_ccg.csv'))

In [7]:
# extract practice statistics for practices that are members of CCGs who are in the RCT
rct_practices = rct_ccgs[["pct_id"]].merge(practice_to_ccg, left_on="pct_id", right_on ="ccg_id", how="left")
# add a new "ccg_id" column just for practices
all_stats_with_ccg = all_stats.merge(
    rct_practices[["ccg_id", "code"]],
    left_on="org_id",
    right_on="code",
    how="left").drop("code", axis=1)
all_stats_with_ccg.loc[all_stats_with_ccg.org_id.str.len() == 3, "ccg_id"] = all_stats_with_ccg.org_id
# Add joint team id and allocation onto the new stats
stats_with_allocations = rct_ccgs.merge(all_stats_with_ccg, left_on="pct_id",right_on="ccg_id",how="left")

In [8]:
# import CCG population sizes

query = '''select prac.ccg_id AS pct_id, sum(stats.total_list_size) as list_size
from `hscic.practice_statistics_all_years` as stats
left join research.practices_2019_09 prac ON stats.practice = prac.code
where CAST(stats.month AS DATE) = '2018-08-01'
group by pct_id
'''
pop = pd.read_gbq(query, GBQ_PROJECT_ID, dialect='standard')
pop.to_csv(os.path.join('..','data','practice_statistics.csv'))

In [9]:
# merge rct_ccgs with population data
ccg_populations = rct_ccgs.merge(pop, on="pct_id", how="left")[["joint_id", "list_size"]]

# group up to joint teams
joint_team_populations = ccg_populations.groupby("joint_id").sum().reset_index()
joint_team_populations.head()


,joint_id,list_size
0,00J,258773
1,00Y,256146
2,01F,131772
3,01J,165657
4,01V,280649


In [10]:
# import dates of interventions
visit_dates = pd.read_csv(os.path.join('..','data','allocated_ccgs_visit_timetable.csv'))
visit_dates["date"] = pd.to_datetime(visit_dates.date)

# merge with rct_ccgs/joint teams
allocations_with_dates = rct_ccgs.merge(visit_dates, on="joint_id", how="left").drop("pct_id", axis=1).drop_duplicates()
allocations_with_dates_and_sizes = allocations_with_dates.merge(joint_team_populations, on="joint_id")

# rank by size, to allow us to pair similar interventions and controls
allocations_with_dates_and_sizes["size_rank"] = allocations_with_dates_and_sizes.groupby("allocation").list_size.rank()

# assign dummy intervention dates to control practices by pairing on total list size
i_group = allocations_with_dates_and_sizes[["allocation", "date", "size_rank"]]\
          .loc[allocations_with_dates_and_sizes.allocation == "I"]\
          .drop("allocation", axis=1)

allocations_with_dates_and_sizes = allocations_with_dates_and_sizes.merge(i_group, on="size_rank", how="left", suffixes=["", "_int"])\
         .drop("date", axis=1)\
         .sort_values(by=["size_rank", "allocation"])
allocations_with_dates_and_sizes.head()

,joint_id,allocation,intervention,list_size,size_rank,date_int
39,10D,I,1,113816,1.0,2018-11-21
14,02G,con,0,113244,1.0,2018-11-21
23,03E,I,1,163530,2.0,NaT
5,01F,con,0,131772,2.0,NaT
16,10V,I,1,215675,3.0,2018-12-12


In [11]:
# join joint-group / ccg allocations, visit dates and list size info to page views data
all_data = allocations_with_dates_and_sizes.drop("size_rank", axis=1)\
       .merge(
           stats_with_allocations.drop(["allocation", "pct_id", "ccg_id", "intervention"], axis=1),
           how='left',
           on='joint_id')
all_data.head(2)

,joint_id,allocation,intervention,list_size,date_int,Page,Date,Pageviews,Unique Pageviews,org_id,org_type
0,10D,I,1,113816,2018-11-21,/ccg/10D/measures/?tags=lowpriority,2018-11-21,10,3,10D,ccg
1,10D,I,1,113816,2018-11-21,/ccg/10D/measures/?tags=lowpriority,2018-11-26,2,2,10D,ccg


In [12]:
# assign each page view occurrence to before vs after intervention (1 month ~ 28 days)

all_data["datediff"] = all_data.Date - all_data.date_int
all_data["timing"] = "none"
all_data.loc[(all_data.datediff <= "28 days") & (all_data.datediff > "0 days"),
      "timing"] = "after"
all_data.loc[(all_data.datediff >= "-28 days") & (all_data.datediff < "0 days"),
      "timing"] = "before"
all_data["Unique Pageviews"] = all_data["Unique Pageviews"].fillna(0)
all_data.head(2)

,joint_id,allocation,intervention,list_size,date_int,Page,Date,Pageviews,Unique Pageviews,org_id,org_type,datediff,timing
0,10D,I,1,113816,2018-11-21,/ccg/10D/measures/?tags=lowpriority,2018-11-21,10,3,10D,ccg,0 days,none
1,10D,I,1,113816,2018-11-21,/ccg/10D/measures/?tags=lowpriority,2018-11-26,2,2,10D,ccg,5 days,after


In [13]:
# group up page views data to joint teams and sum page views before
# and after interventions

all_data_agg = all_data.groupby(["intervention", "joint_id", "org_type", "list_size", "timing"])\
      .agg({"Unique Pageviews": sum, "Page": "nunique"}).unstack().unstack(2).fillna(0).stack()
# added an extra unstack and stack here to fill nulls with zero and ensure each joint_id always counted in its allocation group
#even if it does not have any page views in either the ccg or practice org_type

all_data_agg = all_data_agg.rename(columns={"Page": "No_of_Pages"}).reset_index()
#flatten columns and drop superfluous columns
all_data_agg.columns = all_data_agg.columns.map('_'.join).map(lambda x: x.strip("_"))
all_data_agg = all_data_agg.drop(["Unique Pageviews_none","No_of_Pages_none"], axis=1)
all_data_agg.head()

,intervention,joint_id,list_size,org_type,Unique Pageviews_after,Unique Pageviews_before,No_of_Pages_after,No_of_Pages_before
0,0,00Y,256146,ccg,3.0,0.0,1.0,0.0
1,0,00Y,256146,practice,0.0,1.0,0.0,1.0
2,0,01F,131772,ccg,0.0,0.0,0.0,0.0
3,0,01F,131772,practice,0.0,0.0,0.0,0.0
4,0,01J,165657,ccg,1.0,0.0,1.0,0.0


## Engagement outcome E1
Number of page views over one month on CCG pages showing low-priority measures, before vs after intervention, between intervention and control groups.



In [14]:
# filter CCG page views only:
ccg_data_agg = all_data_agg.loc[all_data_agg.org_type == "ccg"]
ccg_data_agg_trimmed = trim_5_percentiles(ccg_data_agg, debug=False)

display(ccg_data_agg_trimmed.groupby("intervention").agg({"joint_id":"nunique",
                                                            "Unique Pageviews_before":{"sum","mean","std"},
                                                            "Unique Pageviews_after":{"sum","mean","std"}}))

formula = ('data["proxy_pageviews_after"] '
           ' ~ data["proxy_pageviews_before"] + intervention')
compute_regression(
    ccg_data_agg_trimmed,
    formula=formula)

joint_id Unique Pageviews_before                  \
              nunique                     sum       std  mean   
intervention                                                    
0                  20                     2.0  0.307794  0.10   
1                  20                    13.0  1.268028  0.65   

             Unique Pageviews_after                  
                                sum       std  mean  
intervention                                         
0                               8.0  0.753937  0.40  
1                              13.0  1.136708  0.65

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     data["proxy_pageviews_after"]   R-squared:                       0.089
Model:                                       OLS   Adj. R-squared:                  0.039
Method:                            Least Squares   F-statistic:                     1.800
Date:                           Thu, 19 Nov 2020   Prob (F-statistic):              0.179
Time:                                   16:33:50   Log-Likelihood:                -43.138
No. Observations:                             40   AIC:                             92.28
Df Residuals:                                 37   BIC:                             97.34
Df Model:                                      2                                         
Covariance Type:                       nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          0.3199      0.167      1.921      0.062      -0.017       0.657
data["proxy_pageviews_before"]     0.3257      0.192      1.694      0.099      -0.064       0.715
intervention                       0.0697      0.246      0.283      0.779      -0.429       0.569
==============================================================================
Omnibus:                        9.071   Durbin-Watson:                   2.213
Prob(Omnibus):                  0.011   Jarque-Bera (JB):                8.882
Skew:                           1.149   Prob(JB):                       0.0118
Kurtosis:                       3.216   Cond. No.                         2.89
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Engagement outcome E2
Number of page views over one month on practice pages showing low-priority measures, grouped up to CCGs

In [15]:
practice_data_agg = all_data_agg.loc[all_data_agg.org_type == "practice"]
practice_data_agg_trimmed = trim_5_percentiles(practice_data_agg, debug=False)

display(practice_data_agg_trimmed.groupby("intervention").agg({"joint_id":"nunique",
                                                        "Unique Pageviews_before":{"sum","mean","std"},
                                                        "Unique Pageviews_after":{"sum","mean","std"}}))

compute_regression(
    practice_data_agg_trimmed,
    formula=formula)

joint_id Unique Pageviews_before                  \
              nunique                     sum       std  mean   
intervention                                                    
0                  20                     9.0  0.998683  0.45   
1                  20                    44.0  4.212763  2.20   

             Unique Pageviews_after                 
                                sum       std mean  
intervention                                        
0                              12.0  1.142481  0.6  
1                              20.0  1.716790  1.0

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     data["proxy_pageviews_after"]   R-squared:                       0.308
Model:                                       OLS   Adj. R-squared:                  0.271
Method:                            Least Squares   F-statistic:                     8.234
Date:                           Thu, 19 Nov 2020   Prob (F-statistic):            0.00110
Time:                                   16:33:50   Log-Likelihood:                -52.925
No. Observations:                             40   AIC:                             111.9
Df Residuals:                                 37   BIC:                             116.9
Df Model:                                      2                                         
Covariance Type:                       nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          0.4237      0.216      1.964      0.057      -0.014       0.861
data["proxy_pageviews_before"]     0.3919      0.097      4.023      0.000       0.194       0.589
intervention                      -0.2045      0.312     -0.655      0.517      -0.837       0.428
==============================================================================
Omnibus:                       12.733   Durbin-Watson:                   2.172
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               12.819
Skew:                           1.286   Prob(JB):                      0.00165
Kurtosis:                       4.038   Cond. No.                         4.59
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Engagement outcomes E3 and E4 : Alert sign-ups


## Prepare data

In [16]:
# import data from django administration, filtered for confirmed sign-ups only (no date filter)

alerts = pd.read_csv(os.path.join('..','data','orgbookmarks-2019-04-30.csv'))
alerts["created_at"] = pd.to_datetime(alerts.created_at).dt.strftime('%Y-%m-%d')

alerts = alerts.loc[alerts["approved"]=="t"] 
## added this extra step to filter for confirmed sign ups as download from archive is not pre-filtered

alerts = alerts.rename(columns={"pct_id":"pct", "practice_id":"practice", "user_id":"user"})
alerts.head()


,id,pct,practice,user,created_at,approved
0,2989,NaN,Y00028,1288,2019-04-30,t
1,2988,NaN,N83015,2819,2019-04-30,t
2,2987,06M,NaN,3409,2019-04-30,t
3,2986,NaN,L81039,3408,2019-04-30,t
4,2985,NaN,E83009,3406,2019-04-29,t


In [17]:
# map practices to joint teams (and thus only include RCT subjects)
alerts = alerts.merge(
    rct_practices[["ccg_id", "code"]],
    left_on="practice",
    right_on="code",
    how="left").drop("code",axis=1)
# Fill nulls in ccg_id column from values in pct colume
alerts.ccg_id = alerts.ccg_id.combine_first(alerts.pct)
alerts.head()

,id,pct,practice,user,created_at,approved,ccg_id
0,2989,NaN,Y00028,1288,2019-04-30,t,NaN
1,2988,NaN,N83015,2819,2019-04-30,t,01J
2,2987,06M,NaN,3409,2019-04-30,t,06M
3,2986,NaN,L81039,3408,2019-04-30,t,NaN
4,2985,NaN,E83009,3406,2019-04-29,t,NaN


In [18]:
# Add RCT allocations to data
alerts = rct_ccgs.merge(alerts, left_on="pct_id", right_on="ccg_id", how="left")
# flag whether each alert is a practice or CCG alert
conditions = [
    (alerts.pct.str.len()==3),
    (alerts.practice.str.len()==6)]

choices = ['ccg', 'practice']
alerts['org_type'] = np.select(conditions, choices, default='none')
alerts.head()

,joint_id,allocation,pct_id,intervention,id,pct,practice,user,created_at,approved,ccg_id,org_type
0,01X,con,01X,0,2772,01X,NaN,3190,2019-02-15,t,01X,ccg
1,01X,con,01X,0,2477,NaN,N83020,2954,2018-06-12,t,01X,practice
2,01X,con,01X,0,2339,NaN,N83027,2821,2018-05-11,t,01X,practice
3,01X,con,01X,0,1615,01X,NaN,2042,2018-04-27,t,01X,ccg
4,01X,con,01X,0,1262,01X,NaN,1589,2018-01-24,t,01X,ccg


In [19]:
# join to visit dates
alerts_with_dates_and_stats = allocations_with_dates_and_sizes\
                              .drop(["size_rank", "allocation", "intervention"],axis=1)\
                              .merge(alerts.drop(["approved"], axis=1),
                                     how='left', on='joint_id')
alerts_with_dates_and_stats.head()

,joint_id,list_size,date_int,allocation,pct_id,intervention,id,pct,practice,user,created_at,ccg_id,org_type
0,10D,113816,2018-11-21,I,10D,1,1905,NaN,Y02506,2444,2018-07-16,10D,practice
1,10D,113816,2018-11-21,I,10D,1,1766,10D,NaN,2243,2018-07-06,10D,ccg
2,10D,113816,2018-11-21,I,10D,1,354,10D,NaN,460,2017-04-20,10D,ccg
3,02G,113244,2018-11-21,con,02G,0,2921,02G,NaN,3344,2019-05-04,02G,ccg
4,02G,113244,2018-11-21,con,02G,0,2778,02G,NaN,3190,2019-02-15,02G,ccg


In [20]:
# assign each page view occurrence to before vs after intervention (1
# month ~ 28 days)
alerts_with_dates_and_stats["created_at"] = pd.to_datetime(alerts_with_dates_and_stats.created_at)
alerts_with_dates_and_stats["datediff"] = (
    alerts_with_dates_and_stats.created_at - alerts_with_dates_and_stats.date_int)
alerts_with_dates_and_stats["timing"] = "none"
# all alerts set up prior to day of intervention will be used as a co-variable:
alerts_with_dates_and_stats.loc[
    (alerts_with_dates_and_stats.datediff < "0 days"),
    "timing"] = "before"

In [21]:
# main outcome: alerts set up within 3 months of intervention:
alerts_with_dates_and_stats.loc[
    (alerts_with_dates_and_stats.datediff >= "0 days") &
    (alerts_with_dates_and_stats.datediff <= "84 days"),
    "timing"] = "after"  # (within 3 months)

In [22]:
# aggregate data: sum alerts before and after intervention for each joint team
alerts_agg = alerts_with_dates_and_stats\
     .groupby(["intervention", "joint_id", "list_size", "timing", "org_type"])\
     .agg({"user": "nunique"})\
     .unstack()\
     .fillna(0)
alerts_agg = alerts_agg.rename(columns={"user": "alerts"}).unstack().reset_index().fillna(0)

# flatten columns:
alerts_agg.columns = alerts_agg.columns.map('_'.join).map(lambda x: x.rstrip("_"))

alerts_agg["list_size_100k"] = alerts_agg["list_size"]/100000
alerts_agg = alerts_agg[
    ["intervention",
     "joint_id",
     "list_size_100k",
     "alerts_ccg_after",
     "alerts_ccg_before",
     "alerts_practice_after",
     "alerts_practice_before"]]



alerts_agg.head()

,intervention,joint_id,list_size_100k,alerts_ccg_after,alerts_ccg_before,alerts_practice_after,alerts_practice_before
0,0,00Y,2.56146,0.0,3.0,0.0,3.0
1,0,01F,1.31772,0.0,0.0,0.0,0.0
2,0,01J,1.65657,0.0,0.0,1.0,1.0
3,0,01V,2.80649,0.0,0.0,1.0,6.0
4,0,01X,1.97820,0.0,0.0,0.0,0.0


In [23]:
# summary data
display(alerts_agg.groupby("intervention")[["joint_id"]].nunique())
alerts_agg.groupby("intervention").agg(["mean","std"])

,joint_id
intervention,
0,20
1,20


list_size_100k           alerts_ccg_after            \
                       mean       std             mean       std   
intervention                                                       
0                  2.570634  1.201193             0.15  0.366348   
1                  3.337198  1.540121             0.35  0.587143   

             alerts_ccg_before           alerts_practice_after            \
                          mean       std                  mean       std   
intervention                                                               
0                         1.20  1.673320                  0.25  0.550120   
1                         0.85  1.308877                  0.45  0.998683   

             alerts_practice_before            
                               mean       std  
intervention                                   
0                              0.95  1.637553  
1                              2.90  3.754296

### E3 Number of registrations to OpenPrescribing CCG email alerts

In [24]:
formula = ('data["alerts_ccg_after"] ~ '
           'data["alerts_ccg_before"] + list_size_100k + intervention')
compute_regression(
    alerts_agg,
    formula=formula)

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     data["alerts_ccg_after"]   R-squared:                       0.141
Model:                                  OLS   Adj. R-squared:                  0.069
Method:                       Least Squares   F-statistic:                     1.968
Date:                      Thu, 19 Nov 2020   Prob (F-statistic):              0.136
Time:                              16:33:51   Log-Likelihood:                -24.969
No. Observations:                        40   AIC:                             57.94
Df Residuals:                            36   BIC:                             64.69
Df Model:                                 3                                         
Covariance Type:                  nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     0.2660      0.191      1.394      0.172      -0.121       0.653
data["alerts_ccg_before"]     0.0728      0.051      1.416      0.165      -0.031       0.177
list_size_100k               -0.0791      0.056     -1.414      0.166      -0.193       0.034
intervention                  0.2861      0.157      1.818      0.077      -0.033       0.605
==============================================================================
Omnibus:                       12.002   Durbin-Watson:                   2.083
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               11.746
Skew:                           1.192   Prob(JB):                      0.00281
Kurtosis:                       4.167   Cond. No.                         9.58
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""


### E4 Number of registrations to OpenPrescribing Practice email alerts grouped up to CCG
(New sign-ups within 3 months of intervention. The CCG registered population and number of sign-ups prior to the intervention will be co-variables.)

In [25]:
formula = ('data["alerts_practice_after"] ~ '
           'data["alerts_practice_before"] + list_size_100k + intervention')
compute_regression(
    alerts_agg,
    formula=formula)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     data["alerts_practice_after"]   R-squared:                       0.205
Model:                                       OLS   Adj. R-squared:                  0.139
Method:                            Least Squares   F-statistic:                     3.099
Date:                           Thu, 19 Nov 2020   Prob (F-statistic):             0.0387
Time:                                   16:33:51   Log-Likelihood:                -42.842
No. Observations:                             40   AIC:                             93.68
Df Residuals:                                 36   BIC:                             100.4
Df Model:                                      3                                         
Covariance Type:                       nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          0.0075      0.281      0.027      0.979      -0.562       0.576
data["alerts_practice_before"]     0.1178      0.042      2.816      0.008       0.033       0.203
list_size_100k                     0.0508      0.088      0.580      0.566      -0.127       0.229
intervention                      -0.0686      0.256     -0.268      0.790      -0.589       0.451
==============================================================================
Omnibus:                       28.295   Durbin-Watson:                   2.470
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               70.434
Skew:                           1.705   Prob(JB):                     5.07e-16
Kurtosis:                       8.535   Cond. No.                         11.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""